In [9]:
import numpy as np
import pandas as pd

# load ravdess.npy , savee.npy and tess.npy
ravdess_data = np.load('ravdess.npy')
savee_data = np.load('savee.npy') 
tess_data = np.load('tess.npy')
crema_data = np.load('crema.npy')

# ravdess_data.shape, savee_data.shape, tess_data.shape
all_data = np.vstack((ravdess_data, savee_data, tess_data, crema_data))
df = pd.DataFrame(all_data, columns=['label', 'gender', 'pathname', 'filename'])
df.head(), df.shape

(  label  gender                 pathname                  filename
 0     n  female  \data\ravdess\Actor_01\  03-01-01-01-01-01-01.wav
 1     n  female  \data\ravdess\Actor_01\  03-01-01-01-01-02-01.wav
 2     n  female  \data\ravdess\Actor_01\  03-01-01-01-02-01-01.wav
 3     n  female  \data\ravdess\Actor_01\  03-01-01-01-02-02-01.wav
 4     c  female  \data\ravdess\Actor_01\  03-01-02-01-01-01-01.wav,
 (12162, 4))